In [ ]:
%%HTML
<style type="text/css">
     
    
div.h2 {
    background-color: #159957;
    background-image: linear-gradient(120deg, #155799, #159957);
    text-align: left;
    color: white;              
    padding:9px;
    padding-right: 100px; 
    font-size: 20px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 40px; 
}
                                     
                                      
body {
  font-size: 12px;
}    
     
                                    
                                      
div.h3 {
    color: #159957; 
    font-size: 18px; 
    margin-top: 20px; 
    margin-bottom:4px;
}
   
                                      
div.h4 {
    color: #159957;
    font-size: 15px; 
    margin-top: 20px; 
    margin-bottom: 8px;
}
   
                                      
span.note {
    font-size: 5; 
    color: gray; 
    font-style: italic;
}
  
                                      
hr {
    display: block; 
    color: gray
    height: 1px; 
    border: 0; 
    border-top: 1px solid;
}
  
                                      
hr.light {
    display: block; 
    color: lightgray
    height: 1px; 
    border: 0; 
    border-top: 1px solid;
}   
    
                                      
table.dataframe th 
{
    border: 1px darkgray solid;
    color: black;
      <table align="left">
    ...
  </table>
    background-color: white;
}
    
                                      
table.dataframe td 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 11px;
    text-align: center;
} 
   
            
                                      
table.rules th 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 11px;
    align: left;
}
       
                                      
table.rules td 
{
    border: 1px darkgray solid;
    color: black;
    background-color: white;
    font-size: 13px;
    text-align: center;
} 
   
                                      
                                      
table.rules tr.best
{
    color: green;
}    
    
                                      
.output { 
    align-items: left; 
}
        
                                      
.output_png {
    display: table-cell;
    text-align: left;
    margin:auto;
}                                          
                                                                    
                                      
                                      
</style>  

In [ ]:
#**********IMPORTANT INFORMATION**************#
# 86b5fa553252dd29bf3a1ee24d514bdd26afc425
# /kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a.json


#------------------------------------------------------------
import os
#------------------------------------------------------------
import json
import re
import requests
#------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")
#------------------------------------------------------------
import math
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
#------------------------------------------------------------
from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#--------------------- NLTK ---------------------------------
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#------------------------------------------------------------
from IPython.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# #------------------------------------------------------------




# ---------------------------------------------------
# list all files under the input directory
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         # print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.
#
#
#textFile = spark.read.text('path/file.txt')
# json_to_df = spark.read.json()
# ---------------------------------------------------
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
# /kaggle/input/CORD-19-research-challenge/2020-03-13/json_schema.txt
# /kaggle/input/CORD-19-research-challenge/2020-03-13/COVID.DATA.LIC.AGMT.pdf
# /kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv
# /kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.readme
# /kaggle/input/CORD-19-research-challenge/2020-03-13/comm_use_subset/comm_use_subset/25621281691205eb015383cbac839182b838514f.json
#
#
#  Row(sha='469ed0f00c09e2637351c9735c306f27acf3aace', source_x='CZI', title='First two\xa0months of the 2019 Coronavirus Disease (COVID-19) epidemic in China: real-time surveillance and evaluation with a second derivative model', doi='10.1186/s41256-020-00137-4', pmcid=None, pubmed_id=None, license='cc-by', abstract='Similar to outbreaks of many other infectious diseases, success in controlling the novel 2019 coronavirus infection requires a timely and accurate monitoring of the epidemic, particularly during its early period with rather limited data while the need for information increases explosively.', publish_time='2020', authors='Chen, Xinguang; Yu, Bin', journal='Global Health Research and Policy', Microsoft Academic Paper ID='3006645647', WHO #Covidence='#5595', has_full_text='True')



#------------------------------------------------------------
# install pyspark, need internet 
#------------------------------------------------------------
! pip install pyspark 
#------------------------------------------------------------
# pyspark in /opt/conda/lib/python3.6/site-packages (2.4.5)
# py4j==0.10.7 in /opt/conda/lib/python3.6/site-packages (from pyspark) (0.10.7)
#------------------------------------------------------------

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')




In [ ]:
# ----------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import * 

# StructField, StructType, StringType, MapType, IntegerType, etc etc 

# print("Initiate Apache Spark session...")
spark = SparkSession.builder \
                     .master("local[*]") \
                     .appName("COVID-19-Technical-Analysis") \
                     .getOrCreate() 


# create paths for ease 
# * eventually we can expect a different data versus 2020-03-13 *
# * update:  was in fact updated: * 
# Update this source structure as time passes: 
master_file_path = '/kaggle/input/CORD-19-research-challenge'
meta_file_loc = f'{master_file_path}/metadata.csv'
# core input:  /kaggle/input/CORD-19-research-challenge/metadata.csv


# --- read in the metadata csv file, you will want to infer the header ---
sdf = spark.read.csv(meta_file_loc, 
                          header=True, 
                          inferSchema=True)
                          # pyspark.sql.dataframe.DataFrame

    
# rename columns for clarity:  
sdf = sdf.withColumnRenamed("source_x","src").withColumnRenamed("publish_time","published_date").withColumnRenamed('Microsoft Academic Paper ID',"microsoft_paper_id").withColumnRenamed('WHO #Covidence',"WHO_covidence") 


# myschema = StructType([
#     StructField("ID_DAY", DateType()),
#     StructField("SID_STORE", IntegerType()),
#     StructField("NB_TICKET", IntegerType()),
#     StructField("F_TOTAL_QTY", IntegerType()),
#     StructField("F_VAL_SALES_AMT_TCK", DoubleType()),
#     StructField("SITE_FORMAT", StringType())])


spark
# ---

<a id='bkground'></a>
<div class="h2"><i>NG-EDA</i></div>
<div class="h3">Exploratory Data Analysis and Examination:</div>
<div class="h3"><i>COVID-19 Open Research Dataset (CORD-19)</i></div>
<br>

<div class="h3"><i>About Me:</i></div>
* **Name:** &ensp; &ensp; &nbsp; &nbsp; Tom Bresee
* **Location:** &nbsp;&nbsp; &nbsp;  Frisco, TX, USA

<div class="h3"><i>Approach:</i></div>
* Emphasis on _extremely_ clear visualization
* New and innovative approaches to examining our data
* Get beyond surface analysis and probe for true insights into this data 
* Practical data science
* Utilization of **Apache Spark** as core filtering mechanism
  * Thus most of our data will be in *pyspark.sql.dataframe.DataFrame* type format
  * Need to do deep dive via NLP/NLTK
  * Working directly with Apache Spark is not simple (it can get quite complex and has a fairly steep learning curve), it is quite a bit more powerful than pandas, etc, but it is worth the effort, as we will gain added flexibility down the line if our dataset also increases exponentially...
 

<div class="h3"><i>Subject Matter Domain/Expertise:</i></div>
* This data contains references to medical journals and text
 

<div class="h3"><i>Our Directories and Files:</i></div>
<br>
**main directories:**  
&ensp; biorxiv_medrxiv/  
&ensp;  comm_use_subset/  
&ensp;  custom_license/  
&ensp;  noncomm_use_subset/  


**root files: **   
&ensp; &ensp; COVID.DATA.LIC.AGMT.pdf  
&ensp; &ensp; json_schema.txt  
&ensp; &ensp; metadata.csv  
&ensp; &ensp; metadata.readme  


<div class="h3"><i>Metadata (metadata.csv):</i></div> 
Let's examine the `metadata.csv` master file (previously named `all_sources_metadata_2020-03-13.csv`,   
but was updated 3-20-2020) that is included in our datasets  
&ensp;  -  Renamed a few columns for clarity   
&ensp;  -  A random data record sample from that file is shown below: 

In [ ]:
# single_data_record_example = meta_sdf.first().asDict()
# temp_df = pd.DataFrame(single_data_record_example.items())
# pd.DataFrame(meta_sdf.first().asDict().items()).style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
# temp_df = pd.DataFrame(single_data_record_example.items())
# temp_df.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
# temp_df.T. \
# style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
# -----------------------------------------------------------
# >keep>  d = sdf.first().asDict()



# ['sha',
#  'src',
#  'title',
#  'doi',
#  'pmcid',
#  'pubmed_id',
#  'license',
#  'abstract',
#  'published_date',
#  'authors',
#  'journal',
#  'microsoft_paper_id',
#  'WHO_covidence',
#  'has_full_text',
#  'full_text_file']




# Find this one in the meta: /kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a.json
# a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a
# a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a



#sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').asDict
# issues:   d = sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').asDict


# tempdf = pd.DataFrame(d.items(), columns=['Key', 'Value'])
# tempdf.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide_index()

#     type(sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').collect()  # is a list 
#     type(sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a'))  pyspark.sql.dataframe.DataFrame
#     type(sdf.collect()[43651])  pyspark.sql.types.Row



# sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').show()
# +--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+-------------+-------------+---------------+
# |                 sha|src|               title|                 doi|            abstract|published_date|             authors|      journal|has_full_text| full_text_file|
# +--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+-------------+-------------+---------------+
# |a137eb51461b4a4ed...|PMC|The effect of inh...|10.1186/s12918-01...|BACKGROUND: The c...|   2016 Sep 23|McDermott, Jason ...|BMC Syst Biol|         True|comm_use_subset|
# +--------------------+---+--------------------+--------------------+--------------------+--------------+--------------------+-------------+-------------+---------------+




# READ READ READ 
# sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').collect()[0].asDict()  # this is a [Row(sha='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a',
# {'sha': 'a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a',
#  'src': 'PMC',
#  'title': 'The effect of inhibition of PP1 and TNFα signaling on pathogenesis of SARS coronavirus',
#  'doi': '10.1186/s12918-016-0336-6',
#  'abstract': 'BACKGROUND: The complex interplay between viral replication and host immune response during infection remains poorly understood. While many viruses are known to employ anti-immune strategies to facilitate their replication, highly pathogenic virus infections can also cause an excessive immune response that exacerbates, rather than reduces pathogenicity. To investigate this dichotomy in severe acute respiratory syndrome coronavirus (SARS-CoV), we developed a transcriptional network model of SARS-CoV infection in mice and used the model to prioritize candidate regulatory targets for further investigation. RESULTS: We validated our predictions in 18 different knockout (KO) mouse strains, showing that network topology provides significant predictive power to identify genes that are important for viral infection. We identified a novel player in the immune response to virus infection, Kepi, an inhibitory subunit of the protein phosphatase 1 (PP1) complex, which protects against SARS-CoV pathogenesis. We also found that receptors for the proinflammatory cytokine tumor necrosis factor alpha (TNFα) promote pathogenesis, presumably through excessive inflammation. CONCLUSIONS: The current study provides validation of network modeling approaches for identifying important players in virus infection pathogenesis, and a step forward in understanding the host response to an important infectious disease. The results presented here suggest the role of Kepi in the host response to SARS-CoV, as well as inflammatory activity driving pathogenesis through TNFα signaling in SARS-CoV infections. Though we have reported the utility of this approach in bacterial and cell culture studies previously, this is the first comprehensive study to confirm that network topology can be used to predict phenotypes in mice with experimental validation. ELECTRONIC SUPPLEMENTARY MATERIAL: The online version of this article (doi:10.1186/s12918-016-0336-6) contains supplementary material, which is available to authorized users.',
#  'published_date': '2016 Sep 23',
#  'authors': 'McDermott, Jason E.; Mitchell, Hugh D.; Gralinski, Lisa E.; Eisfeld, Amie J.; Josset, Laurence; Bankhead, Armand; Neumann, Gabriele; Tilton, Susan C.; Schäfer, Alexandra; Li, Chengjun; Fan, Shufang; McWeeney, Shannon; Baric, Ralph S.; Katze, Michael G.; Waters, Katrina M.',
#  'journal': 'BMC Syst Biol',
#  'has_full_text': 'True',
#  'full_text_file': 'comm_use_subset'}


# new - keep:
d = sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').collect()[0].asDict()
tempdf = pd.DataFrame(d.items(), columns=['Key', 'Value'])
tempdf.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide_index()



#sdf.collect()[43651]
# working original - keep: 
# d = sdf.collect()[43651].asDict()
# tempdf = pd.DataFrame(d.items(), columns=['Key', 'Value'])
# tempdf.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide_index()

<div class="h4"><i> - Metafile Header Breakout:</i></div>
**sha**  
&ensp;&ensp; Secure Hash Algorithm (SHA) hash key (not applicable for our analysis)  
&ensp;&ensp; 17K of the records have PDFs and the **hash** of the PDFs are in this column  
&ensp;&ensp; SHA-1 encryption calculates a numeric footprint of 40 hexadecimal characters


**src**  
&ensp;&ensp; Source:  Either `PMC` (PubMed Central), `CZI` (Chan Zuckerberg Initiative), `bioRxiv`,   
&ensp;&ensp; `Elsevier`, `medRxiv`, or `WHO` 

**title**  
&ensp;&ensp; Actual Document Title (title of the work/publication)
 
**doi**  
&ensp;&ensp; Digital Object Identifier  (almost always starts with 10.)  
&ensp;&ensp; String of numbers/letters/symbols to permanently identify article    
&ensp;&ensp; In theory, if they are truly unique identifiers, will allow to determine total doc unique count
 
**pmcid**  
&ensp;&ensp; pmcid, populated for all PMC paper records (example: `PMC3093355`)

**pubmed_id**  
&ensp;&ensp;  pubmet id (example: `32125455`) 
 
**license**  
&ensp;&ensp; License
 
**abstract**  
&ensp;&ensp; Paper abstract 
 
**publish_time**  
&ensp;&ensp; Publish Time (example: `2019 Nov 11`)  
&ensp;&ensp; Note:  Some of the articles are labelled with only the year (`2020`)

**authors**  
&ensp;&ensp; Name of Author(s)

**journal**  
&ensp;&ensp; Journal Name  
 
**Microsoft Academic Paper ID**  
&ensp;&ensp; Microsoft Academic Paper ID (populated for some of the records)
 
**WHO #Covidence**  
&ensp;&ensp; WHO #Covidence (example: `#5386`)

**has_full_text**  
&ensp;&ensp; Does .json have full text ? (`True` or `False`)

**full_text_file (new)**   
&ensp;&ensp; Signals the tar.gz file in which the full text json resides





<div class="h4"><i> - Observations:</i></div>
* 28,450 of the entries have `sha` populated
* The `sha` entry **may** contain more than a single sha entry, i.e. it may have sha1 ; sha2 (separated by a `;`)

* The original metadata column `publish_time` is a bit confusing, I renamed it to `published_date`, as this is a physical date of publication and not a time per se. People will understand published date verbage easier. 
* 96% of the `WHO #Covidence` column data is missing, will delete this column
* 98% of the `Microsoft Academic Paper ID` column data is missing (or not applicable), will delete this column 
* 40,750 of the entries have a `doi` populated
* 42,230 unique paper titles 
* If one needs to sort based on the publication date, will need to convert the `published_date` to datetime format, as it defaults to string...
 

In [ ]:

# ['sha',
#  'src',
#  'title',
#  'doi',
#  'pmcid',
#  'pubmed_id',
#  'license',
#  'abstract',
#  'published_date',
#  'authors',
#  'journal',
#  'microsoft_paper_id',
#  'WHO_covidence',
#  'has_full_text',
#  'full_text_file']



# Streamline our spark data frame (sdf): 
sdf = sdf.drop('pmcid', 'pubmed_id', 'license', 'WHO_covidence', 'microsoft_paper_id')



# delete more ! 
# sdf.columns
# --- output --- 
# ['sha',
#  'src',
#  'title',
#  'doi',
#  'abstract',
#  'published_date',
#  'authors',
#  'journal',
#  'has_full_text',
#  'full_text_file']

<div class="h4"><i> - Summary of metadata file (for updated dataset):</i></div>
<br>
&ensp; **Number of raw record entries:** &ensp; 44,230    
&ensp; **Number of unique journals quoted:** &ensp; 3,945  
&ensp; **Number of populated sha entries:** &ensp; 28,450

In [ ]:

#sdf.select('title').show()

tdf = spark.read.csv(meta_file_loc, 
                          header=True, 
                          inferSchema=True)    
# rename columns for clarity:  
tdf = tdf.withColumnRenamed("source_x","src").withColumnRenamed("publish_time","published_date").withColumnRenamed('Microsoft Academic Paper ID',"microsoft_paper_id").withColumnRenamed('WHO #Covidence',"WHO_covidence") 
tdf = tdf.drop('pmcid', 'pubmed_id', 'license', 'WHO_covidence', 'microsoft_paper_id')

# tdf.show()
# +--------------------+--------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------------+
# |                 sha|     src|               title|                 doi|            abstract|published_date|             authors|             journal|has_full_text|full_text_file|
# +--------------------+--------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-------------+--------------+
# |                null|Elsevier|Intrauterine viru...|10.1016/0002-8703...|Abstract The etio...|    1972-12-31|   Overall, James C.|American Heart Jo...|        False|custom_license|
# |                null|Elsevier|Coronaviruses in ...|10.1016/0002-8703...|                null|    1980-03-31|Georgescu, Leonid...|American Heart Jo...|        False|custom_license|
# |                null|Elsevier|Cigarette smoking...|10.1016/0002-8703...|                null|    1980-03-31|    Friedman, Gary D|American Heart Jo...|        False|custom_license|



# keep and use 
import pyspark.sql.functions as F
# new col with length of string of title (number of characters)
tdf.withColumn('NewC', F.length('title')).show()



<br><hr><br>

<div class="h3"><i>Examining JSON Files</i></div>

* Let's look at the raw json data files 

<div class="h4"><i> - Actual JSON files:</i></div>

* **Total**:  29,315 unique .json files, spread among four subsets
  * **1 - custom_license**: 16959 full text (new: 15533)
  * **2 - noncomm_use_subset**: 2353 full text (new: 385)
  * **3 - comm_use_subset**: 9118 full text (new: 128)
  * **4 -biorxiv_medrxiv**: 885 full text (new: 110)
  * I've created a full list of the dataset .json files =>  &ensp; [here](https://raw.githubusercontent.com/tombresee/COVID-19/master/ENTER/all.txt)
    * READ:  Every .json file is named by its sha hash key ! 

 


<div class="h4"><i> -  Spark Schema:</i></div>

  * Read my .json Spark inferred schema =>  &ensp; [here](https://raw.githubusercontent.com/tombresee/COVID-19/master/ENTER/spark_json_schema_sample.txt)



In [ ]:

#/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a.json

example_json_file_path = '/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a.json'
sample = spark.read.json(example_json_file_path, multiLine=True)
# sample.show(100, False) 
# >keep>   sample.printSchema()
# sample.show(100)
# |            abstract|         back_matter|         bib_entries|           body_text|            metadata|            paper_id|         ref_entries|


<div class="h4"><i> -  Deconstructing a <u>single</u> json file sample to get a feel for the data:</i></div>
<br>

I have found the correlated .json file associated with the metadata.csv entry we examined above (i.e.  
`The effect of inhibition of PP1 and TNFα signaling on pathogenesis of SARS coronavirus`).    
We will decomposed this .json file so we can extract text, etc.  

In other words:  We have previously examined the meta file, found a set entry, now we will actually examine the core json file entry associated with that metadata entry.  Thus we will be able to see how the metadata file had 'meta' information about the json 'paper' file...



Deconstructing Apache Spark dataframe elements:

In [ ]:

d = sample.collect()[0].asDict()
tempdf = pd.DataFrame(d.items(), columns=['Key', 'Value'])
tempdf.style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide_index()



Let's parse thru some of the exact specifics of what we are looking for ... 

Paper ID:  
Recognize this id ?  It is the sha entry from the metafile entry above :) 

In [ ]:
for element in sample.collect():
    print(element.paper_id)
    

# sample.columns
# ['abstract',
#  'back_matter',
#  'bib_entries',
#  'body_text',
#  'metadata',
#  'paper_id',
#  'ref_entries']
    
    

One of the Authors full name: 

In [ ]:
for element in sample.collect():
    print(element.metadata.authors[1].first, element.metadata.authors[1].middle[0], element.metadata.authors[1].last)

Title:  (matches the name in our first table above)

In [ ]:
for element in sample.collect():
    print(element.metadata.title)

Abstract:

In [ ]:

for element in sample.collect():
    print(element.abstract[0].text)
    

In [ ]:

for element in sample.collect():
    print(element.abstract[1].text)
    

Body Text (single snippet of many):

In [ ]:

for element in sample.collect():
    print(element.body_text[0].text)
    

Now we can extract precisely which text we want, based on a fundamental dictionary-like Spark dataframe

Ultimately we do not care much about the excessive data such as BIB entries and ref_entries in the json files, we care way more about the core elements such as Title, Abstract, Body of Text, Article Text, etc

In [ ]:
#----------------------------------------------------
# dont care and dont need: 

# for element in sample.collect():
#     print(element.bib_entries)
    
    
# #     for sub in element.bib_entries:
# #         print(sub)
#----------------------------------------------------



# df.rdd.map(lambda row: row.asDict())



    
# sample.collect()[0].asDict()
# for k in sample.collect()[0].asDict():
#     print(k[1].text)
    
    
    


# sdf.where(sdf.sha=='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a').collect()[0].asDict()  # this is a [Row(sha='a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a',
 

# ===================
# follow up here: *** 
# ===================
# KEEPER !  KEEP  

# df_list_of_dict = [row.asDict() for row in sample.collect()]
# df_list_of_dict[0] 
# {'abstract': [Row(cite_spans=[], ref_spans=[], section='Abstract', text='Background: The complex interplay between viral replication and host immune response during infection remains poorly understood. While many viruses are known to employ anti-immune strategies to facilitate their replication, highly pathogenic virus infections can also cause an excessive immune response that exacerbates, rather than reduces pathogenicity. To investigate this dichotomy in severe acute respiratory syndrome coronavirus (SARS-CoV), we developed a transcriptional network model of SARS-CoV infection in mice and used the model to prioritize candidate regulatory targets for further investigation.'),
#   Row(cite_spans=[], ref_spans=[], section='Abstract', text='Results: We validated our predictions in 18 different knockout (KO) mouse strains, showing that network topology provides significant predictive power to identify genes that are important for viral infection. We identified a novel player in the immune response to virus infection, Kepi, an inhibitory subunit of the protein phosphatase 1 (PP1) complex, which protects against SARS-CoV pathogenesis. We also found that receptors for the proinflammatory cytokine tumor necrosis factor alpha (TNFα) promote pathogenesis, presumably through excessive inflammation. Conclusions: The current study provides validation of network modeling approaches for identifying important players in virus infection pathogenesis, and a step forward in understanding the host response to an important infectious disease. The results presented here suggest the role of Kepi in the host response to SARS-CoV, as well as inflammatory activity driving pathogenesis through TNFα signaling in SARS-CoV infections. Though we have reported the utility of this approach in bacterial and cell culture studies previously, this is the first comprehensive study to confirm that network topology can be used to predict phenotypes in mice with experimental validation.')],

    

<div class="h4"><i> - Condensation:</i></div>

* Thus, we have taken a single json sample like [this](https://raw.githubusercontent.com/tombresee/COVID-19/master/ENTER/json_sample.txt), and are now able to parse through specifics of the json file to get to what we really want, i.e. [this](https://raw.githubusercontent.com/tombresee/COVID-19/master/ENTER/condensed.txt)



<div class="h4"><i> - Observations:</i></div>
* I find it surprising how much of the json file is taken up by Bibliographic References  (bib entries) and figure references
 

<div class="h4"><i> - Extensions:</i></div>
* Now that we have a core approach for building out the json files to Apache Spark Dataframes, we can also if needed/required **export all of the raw data to a remote MongoDB database**, using [this](https://www.mongodb.com/products/spark-connector) core connector

* And thus we can extend a [notebook](https://github.com/tombresee/Prairie-Fire/blob/master/ENTER/ApacheSparkMongoConnector.ipynb) I created for exporting Spark information directly to a Mongo database.  Yet another added benefit of using our Spark-based approach, as we can leverage the flexibility that Mongo has especially with unstructured data... 





<div class="h4"><i> - Next Steps:</i></div>

* Now that we have an algorithm for breaking out the json files to their text particulars, we will parse through all of the titles, article text, and medical verbage 
* Generate word-counts, noun phrases, sentiment, etc, which Apache Spark is spectacular at...

<br><hr><br>

<div class="h3"><i>NLP:  Word Counts</i></div>

Let's start examining words

In [ ]:
#  Generating word counts -  NEEDS WORK 

# lines = tdf.select('title').collect()
# #print(lines)


# words = lines.flatMap(lambda line: line.split(" "))
# word_counts = words.map(lambda word: (word,1))
# word_counts



# tdf.select('title').show(truncate=False)
# tdf.select('title').show(n=12, truncate=False, vertical=True) 


# words = lines.flatMap(lambda line: line.split(" "))
# word_counts = words.map(lambda word: (word,1))



<div class="h4"><i> - Metadata File:</i></div>
* Let's start by going back to the metadata.csv file and breaking out word counts

... more to follow, I'm just getting started ...

<br><br><br><br><br><br><br><br><br><br><br><br>

<div class="alert alert-warning">
<br>
<b>Warning:</b>  &ensp; Medical information enclosed in the dataset files may sometimes have typos, mistakes, incorrect entries, or entries that were meant to be inserted into other columns.  It is important when filtering to keep your eye on this. 
<br><br>

<b>Note:</b> &ensp; Reading in .csv files <u>only works well when the source file was created with proper utilization of the `,` as the delimiter</u>.  IF the comma is used to separate authors in a particular way, it will in fact skew the row of data.  It appears the source .csv files in some location did in fact enter author data but instead of using the `;` to separate multiple authors under the author column, `,` was used instead !   
 
 
<br>

</div>

In [ ]:

# # RIFF / RAFF: 


# df.head(1).T. \
# style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])


# meta_sdf.first().asDict()


# # for k,v in tips.items():
# #     print(f'{k}:\t{v}')
# {'sha': 'c630ebcdf30652f0422c3ec12a00b50241dc9bd9',
#  'source_x': 'CZI',
#  'title': 'Angiotensin-converting enzyme 2 (ACE2) as a SARS-CoV-2 receptor: molecular mechanisms and potential therapeutic target',
#  'doi': '10.1007/s00134-020-05985-9',
#  'pmcid': None,
#  'pubmed_id': '32125455',
#  'license': 'cc-by-nc',
#  'abstract': None,
#  'publish_time': '2020',
#  'authors': 'Zhang, Haibo; Penninger, Josef M.; Li, Yimin; Zhong, Nanshan; Slutsky, Arthur S.',
#  'journal': 'Intensive Care Med',
#  'Microsoft Academic Paper ID': '2002765492',
#  'WHO #Covidence': '#3252',
#  'has_full_text': 'True'}



# # meta_sdf.first().asDict()

# # for k,v in meta_sdf.first().items():
# #     print(f'{k}:\t{v}')


# temp = meta_sdf.first().asDict()

# for k,v in temp.items():
#     print(f'{k}:\t{v}')  
    
    
    
# for k,v in tips.items():
#     print(f'{k}:\t{v}')



# meta_sdf.first().asDict()
# {'sha': 'c630ebcdf30652f0422c3ec12a00b50241dc9bd9',
#  'source_x': 'CZI',
#  'title': 'Angiotensin-converting enzyme 2 (ACE2) as a SARS-CoV-2 receptor: molecular mechanisms and potential therapeutic target',
#  'doi': '10.1007/s00134-020-05985-9',
#  'pmcid': None,
#  'pubmed_id': '32125455',
#  'license': 'cc-by-nc',
#  'abstract': None,
#  'publish_time': '2020',
#  'authors': 'Zhang, Haibo; Penninger, Josef M.; Li, Yimin; Zhong, Nanshan; Slutsky, Arthur S.',
#  'journal': 'Intensive Care Med',
#  'Microsoft Academic Paper ID': '2002765492',
#  'WHO #Covidence': '#3252',
#  'has_full_text': 'True'}


# for item in row: print(item)
    
# for item in row.asDict():
#     print(item)

# for item in row.asDict().keys():
#     print(item)
    
    
    
# for item in row.asDict().keys():
# print(item)
    
    
    
    
    
# for item in row: print(item)
    
# for item in row.asDict():
#     print(item)

# for item in row.asDict().keys():
#     print(item)
    
    
    
    
# meta_sdf.first().asDict()

# for k,v in meta_sdf.first().items():
#     print(f'{k}:\t{v}')

# temp = meta_sdf.first().asDict()

# for k,v in temp.items():
#     print(f'{k}:\t{v}')  

# sha:	c630ebcdf30652f0422c3ec12a00b50241dc9bd9
# source_x:	CZI
# title:	Angiotensin-converting enzyme 2 (ACE2) as a SARS-CoV-2 receptor: molecular mechanisms and potential therapeutic target
# doi:	10.1007/s00134-020-05985-9
# pmcid:	None
# pubmed_id:	32125455
# license:	cc-by-nc
# abstract:	None
# publish_time:	2020
# authors:	Zhang, Haibo; Penninger, Josef M.; Li, Yimin; Zhong, Nanshan; Slutsky, Arthur S.
# journal:	Intensive Care Med
# Microsoft Academic Paper ID:	2002765492
# WHO #Covidence:	#3252
# has_full_text:	True
    
    
# meta_sdf.first().asDict()

# for k,v in meta_sdf.first().items():
#     print(f'{k}:\t{v}')

# temp = meta_sdf.first().asDict()

# for k,v in temp.items():
#     print(f'{k}:\t{v}') 
# sha:	c630ebcdf30652f0422c3ec12a00b50241dc9bd9
# source_x:	CZI
# title:	Angiotensin-converting enzyme 2 (ACE2) as a SARS-CoV-2 receptor: molecular mechanisms and potential therapeutic target
# doi:	10.1007/s00134-020-05985-9
# pmcid:	None
# pubmed_id:	32125455
# license:	cc-by-nc
# abstract:	None
# publish_time:	2020
# authors:	Zhang, Haibo; Penninger, Josef M.; Li, Yimin; Zhong, Nanshan; Slutsky, Arthur S.
# journal:	Intensive Care Med
# Microsoft Academic Paper ID:	2002765492
# WHO #Covidence:	#3252
# has_full_text:	True
    
    
# <div class="h4"><i>Let's examine the actual metadata file:</i></div>
# * abc

#----------------------------------------------------------------------------------------------------------------------------------
# SUPERCOUNTS:
#    meta_sdf.select('doi').distinct().count()
#    meta_sdf.select('journal').distinct().count()
#    meta_sdf.select('publish_time').distinct().show()  < - - MAKE A DATA FORM ! 
#    meta_sdf.count()
#    meta_sdf.filter(meta_sdf.has_full_text =="True").count()
#    meta_sdf.select('publish_time').distinct().show()
#    meta_sdf.filter(meta_sdf.has_full_text =="False").count()
#    meta_sdf.count()
#    meta_sdf.select('journal').distinct().show(300,False)
#----------------------------------------------------------------------------------------------------------------------------------
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# /kaggle/input/CORD-19-research-challenge/2020-03-13/json_schema.txt
# /kaggle/input/CORD-19-research-challenge/2020-03-13/COVID.DATA.LIC.AGMT.pdf
# /kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.csv
# /kaggle/input/CORD-19-research-challenge/2020-03-13/all_sources_metadata_2020-03-13.readme
# /kaggle/input/CORD-19-research-challenge/2020-03-13/comm_use_subset/comm_use_subset/25621281691205eb015383cbac839182b838514f.json
#----------------------------------------------------------------------------------------------------------------------------------
#  link to live jupyter:
#    https://kkb-production.jupyter-proxy.kaggle.net/k/30518687/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..MslS0K3Jv0xA9ckYVozGYg.Bh-iEBt7HITa9f1OAw1uQ1dE-Xr_oHQ9_1vYT2tx6CLnuffbRwMHpA1jWHuijs0-zQWj4Ql1NGxbwt9m7kB6LITko_lrWyIMxxToyPH8bK4.RN73ZD5-RWss9nqcYb5e_A/proxy/tree
#
#
#  
#    MY LIVE JUPYTER NOTEBOOK:  ( I CAN SAVE / ACCESS FILES HERE )
#    https://www.kaggle.com/tombresee/ng-eda-covid-19/edit/run/30478508
#  
# 
#
# from IPython.display import FileLink
# FileLink(r'whatswrong/whatswrong')
#
#
# master_file_path = '/kaggle/input/CORD-19-research-challenge/2020-03-13'
# meta_file_loc = f'{master_file_path}/all_sources_metadata_2020-03-13.csv'
# pdf = pd.read_csv(meta_file_loc)
# len(pdf)
#
#
#
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#
#
# from IPython.display import HTML
# import pandas as pd
# import numpy as np


# # df = pd.DataFrame(np.arange(1000), columns=['data'])

# # df.to_csv('submission.csv')

# def create_download_link(title = "Download CSV file", filename = "data.csv"):  
#     html = '<a href={filename}>{title}</a>'
#     html = html.format(title=title,filename=filename)
#     return HTML(html)

# # create a link to download the dataframe which was saved with .to_csv method
# create_download_link(filename='whatswrong.csv')

        
#######################################################################################################################################
#  https://www.kaggle.com/shivamb/interactive-d3-js-visualisations-in-kaggle-kernels#Interactive-D3.js-Visualisations-in-Kaggle-Kernels
#######################################################################################################################################

# h = display(HTML(htmlt1))
# j = IPython.display.Javascript(js_t1)
# IPython.display.display_javascript(j)

#  meta_sdf.describe(['has_full_text']).show()




# (1) Metadata for papers from these sources are combined: CZI, PMC, BioRxiv/MedRxiv. (total records 29500)
# 	- CZI 1236 records
# 	- PMC 27337
# 	- bioRxiv 566
# 	- medRxiv 361
# (2) 17K of the records have PDFs and the hash of the PDFs are in 'sha'
# (3) For PMC sourced papers, one paper's metadata can be associated with one or more PDFs/shas under that paper - a PDF/sha correponding to the main article, and possibly additional PDF/shas corresponding to supporting materials for the article.
# (4)	13K of the PDFs were processed with fulltext ('has_full_text'=True)
# (5) Various 'keys' are populated with the metadata:
# 	- 'pmcid': populated for all PMC paper records (27337 non null)
# 	- 'doi': populated for all BioRxiv/MedRxiv paper records and most of the other records (26357 non null)
# 	- 'WHO #Covidence': populated for all CZI records and none of the other records (1236 non null)
# 	- 'pubmed_id': populated for some of the records
# 	- 'Microsoft Academic Paper ID': populated for some of the records



# 2020-03-20
# ---CHANGES---
# * normalized doi, authors, title, abstract strings
# * merged redundant rows in metadata file on doi, pmcid, and pubmed_id
# * metadata sha column can now include multiple files (some PMC files have multiple associated PDFs)
# * added column in metadata file "full_text_file" to signal the tar.gz file in which the full text json resides
# ---SUMMARY---
# total metadata rows: 44220
# custom_license: 16959 full text (new: 15533)
# noncomm_use_subset: 2353 full text (new: 385)
# comm_use_subset: 9118 full text (new: 128)
# biorxiv_medrxiv: 885 full text (new: 110)
    
    
    
#     Collecting pyspark
#   Downloading pyspark-2.4.5.tar.gz (217.8 MB)
#      |████████████████████████████████| 217.8 MB 4.5 kB/s  eta 0:00:01
# Collecting py4j==0.10.7
#   Downloading py4j-0.10.7-py2.py3-none-any.whl (197 kB)
#      |████████████████████████████████| 197 kB 36.7 MB/s eta 0:00:01
# Building wheels for collected packages: pyspark
#   Building wheel for pyspark (setup.py) ... done
#   Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218258791 sha256=e472a65288dc2910642aabb9c044a1cc9d345adbb23c2a068bb2ddcc44b1fd40
#   Stored in directory: /root/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa
# Successfully built pyspark
# Installing collected packages: py4j, pyspark
# Successfully installed py4j-0.10.7 pyspark-2.4.5



#   --- UPDATED --- 
# import os
# for dirname, _, filenames in os.walk('/kaggle/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
# /kaggle/lib/kaggle/gcp.py
# /kaggle/input/CORD-19-research-challenge/metadata.csv
# /kaggle/input/CORD-19-research-challenge/json_schema.txt
# /kaggle/input/CORD-19-research-challenge/metadata.readme
# /kaggle/input/CORD-19-research-challenge/COVID.DATA.LIC.AGMT.pdf
# /kaggle/input/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset/d99acb4e99be7852aa61a688c9fbd38d44b5a252.json




# (1) Metadata for papers from these sources are combined: CZI, PMC, BioRxiv/MedRxiv. (total records 29500)
# 	- CZI 1236 records
# 	- PMC 27337
# 	- bioRxiv 566
# 	- medRxiv 361
# (2) 17K of the records have PDFs and the hash of the PDFs are in 'sha'
# (3) For PMC sourced papers, one paper's metadata can be associated with one or more PDFs/shas under that paper - a PDF/sha correponding to the main article, and possibly additional PDF/shas corresponding to supporting materials for the article.
# (4)	13K of the PDFs were processed with fulltext ('has_full_text'=True)
# (5) Various 'keys' are populated with the metadata:
# 	- 'pmcid': populated for all PMC paper records (27337 non null)
# 	- 'doi': populated for all BioRxiv/MedRxiv paper records and most of the other records (26357 non null)
# 	- 'WHO #Covidence': populated for all CZI records and none of the other records (1236 non null)
# 	- 'pubmed_id': populated for some of the records
# 	- 'Microsoft Academic Paper ID': populated for some of the records




# 2020-03-20
# ---CHANGES---
# * normalized doi, authors, title, abstract strings
# * merged redundant rows in metadata file on doi, pmcid, and pubmed_id
# * metadata sha column can now include multiple files (some PMC files have multiple associated PDFs)
# * added column in metadata file "full_text_file" to signal the tar.gz file in which the full text json resides
# ---SUMMARY---
# total metadata rows: 44220
# custom_license: 16959 full text (new: 15533)
# noncomm_use_subset: 2353 full text (new: 385)
# comm_use_subset: 9118 full text (new: 128)
# biorxiv_medrxiv: 885 full text (new: 110)


# meta_sdf.filter((meta_sdf.sha)>40).show()




 
#
#
#      COMMANDS: 
#                   meta_sdf.select("sha").show(10,False)
#
#                   meta_sdf.collect()[221].asDict()
#
#                   meta_sdf.select("doi").distinct().show(10,False)
#
#                   meta_sdf.select("sha").filter(lamdba string: len(string)>40).show()
#
#                   meta_sdf.count()
#
#  meta_sdf.select('sha').describe().show(100,False)
# +-------+----------------------------------------+
# |summary|sha                                     |
# +-------+----------------------------------------+
# |count  |28512                                   |
# |mean   |null                                    |
# |stddev |null 




# # meta_sdf.filter(size(col(meta_sdf.sha)) > 40)
# meta_sdf.where(size(col(meta_sdf.sha)) >=40).show()




# add this:
# <br><br><br>
# <div class="h4"><i>External References:</i></div>

# ```meta_sdf.filter(meta_sdf.sha == 'aecbc613ebdab36753235197ffb4f35734b5ca63')```
 
    
    
    
    
# add this ! ! ! ! ! !   and reuse: 
# meta_sdf.select('source_x').distinct().describe().show(100,False)

# sdf.select('license').distinct().show(100,False)

# sdf.select('full_text_file').distinct().show(100,False)

# sdf.select('authors').distinct().show(1000,False)



# issues, follow up: 
# sdf.select('journal').distinct().orderBy(sdf.journal.desc()).show(100, False)

# sdf.select('journal').distinct().orderBy('journal').show(100, False)

# sdf.select('journal').distinct().sort('journal', ascending=True).show(1000, False)   < - - - 

# sdf.select('journal').distinct().sort('journal', ascending=True).count()







#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# counter = 0
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         counter = counter + 1 
#         #print(os.path.join(dirname, filename))
# print(counter)
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++







# ! cat /kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset/a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a.json









#https://stackoverflow.com/questions/49432167/how-to-convert-rows-into-dictionary-in-pyspark







#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#sample.select("metadata").show(100, False)

# df_list_of_dict = [row.asDict() for row in sample.collect()]
# df_list_of_dict


# for i in df_list_of_dict: 
#     print("\n", i)
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    


# HELPER-X
# import pyspark.sql.functions as F
# # new col with length of string of title (number of characters)
# tdf.withColumn('NewC', F.length('title')).show()

























# HELPER-X:  display df pretty 
# display(tdf.select('title').toPandas().style.hide_index())
# HUGE OUTPUT *** 





# # HELPER-X:  html output 
# tempA = pd.DataFrame([100,200,300])
# # from IPython.display import HTML
# # HTML(tempA.to_html(index=False))
# # 0
# # 100
# # 200
# # 300
# #tempA.style.hide_index()






# tdf.show(n=2, truncate=False, vertical=True)
# -RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#  sha            | null                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
#  src            | Elsevier                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#  title          | Intrauterine virus infections and congenital heart disease                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
#  doi            | 10.1016/0002-8703(72)90077-4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
#  abstract       | Abstract The etiologic basis for the vast majority of cases of congenital heart disease remains largely undefined. Viruses have been considered to be likely candidates since the recognition of the association between intrauterine rubella and congenital heart disease. Although the pathogenesis of cardiovascular defects is poorly understood, information gained from the study of congenital rubella syndrome suggests that mechanisms such as focal endothelial cell damage, resulting in obliteration of vascular supply, decreased growth rate, and shortened survival time of certain cells, and disturbed DNA replication in cells whose chromosomes were damaged secondary to the effects of virus replication may be operative in the production of defects in the developing fetus. In addition to rubella there is suggestive, but not conclusive, evidence that Coxsackie B3 and B4 virus infections during pregnancy can result in the birth of infants with a variety of types of congenital heart lesions and that intrauterine mumps virus infection may be etiologically related to the postnatal development of endocardial fibroelastosis (EFE). Although there are a number of other viruses that are potential etiologic agents of congenital heart disease, the current status of information is inadequate to allow even suggestive associations to be made. The most profitable areas for future investigation appear to be: (1) the epidemiology of congenital heart disease, (2) prospective studies of the association of maternal viral infection with abnormal offspring, (3) the in-depth virologic investigation of the infant with a cardiac defect, and (4) the development of experimental animal models of congenital heart disease. Successful control of virus-induced congenital heart disease will depend on the results of these investigations and the development of vaccines against the identified causative viruses and/or safe and effective antiviral chemotherapy for the woman in early gestation who is infected with a known teratogenic agent. 
#  published_date | 1972-12-31                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
#  authors        | Overall, James C.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
#  journal        | American Heart Journal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
#  has_full_text  | False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
#  full_text_file | custom_license                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# -RECORD 1-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#  sha            | null                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
#  src            | Elsevier                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
#  title          | Coronaviruses in Balkan nephritis                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
#  doi            | 10.1016/0002-8703(80)90355-5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
#  abstract       | null                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
#  published_date | 1980-03-31                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
#  authors        | Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan; Plavoşin, Livia; Herzog, Georgeta                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
#  journal        | American Heart Journal                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
#  has_full_text  | False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
#  full_text_file | custom_license                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# only showing top 2 rows

In [ ]:
#  Doing things you may need later:



#************************************************************************************************
# df = df.withColumn('wordCount', f.size(f.split(f.col('Description'), ' ')))
# df.show()
#+-------------------+-----------------+---------+
#|              Dates|      Description|wordCount|
#+-------------------+-----------------+---------+
#|2015-05-14 03:53:00|   WARRANT ARREST|        2|
#|2015-05-14 03:53:00|TRAFFIC VIOLATION|        2|
#|2015-05-14 03:33:00|TRAFFIC VIOLATION|        2|
#+-------------------+-----------------+---------+

df = tdf.withColumn('wordCountfromTitle', F.size(F.split(F.col('title'), ' ')))

df.show()



